In [2]:
import os

import gurobipy as gp
from gurobipy import GRB
import numpy as np

# Defining functions and formulations

In [3]:
# set index r for 2nd and 3rd formulations
# r = np.random.randint(n)
r2 =5 
r3 = 5

In [4]:
def read_data(dir='data'):
    data = dict()
    files = os.listdir(dir)  # Lists all files and folders
    # Filter only files (exclude directories)
    files = [f for f in files if os.path.isfile(os.path.join(dir, f))]
    for f in files:
        name = f.split('.')
        if name[1] == 'txt':
            name = name[0]
            # reading the file
            with open(os.path.join(dir, f), "r") as ff:
                lines = ff.readlines()
            
            # Extract number of nodes
            num_nodes = int(lines[0].strip())
            # Load distance matrix
            distance_matrix = np.loadtxt(lines[1:], dtype=int)
            data[name] = distance_matrix
    return data

In [5]:
def print_results(model:gp.Model):
    if model.status == GRB.OPTIMAL:
        logFile = model.getParamInfo('logFile')[2]
        with open(logFile, 'a') as log:
            print("Optimal Solution:")
            log.write("\n \t Optimal Solution:\n")
            for v in model.getVars():
                if v.X != 0:  # 1 for x and f values and integer numbers for u values
                    print(f'{v.VarName}: {v.X}')
                    log.write(f'\t{v.VarName}: {v.X} \n')
            print(f'objective value: {model.Objval}')
            log.write(f'\tobjective value: {model.Objval}')
    else:
        print("No optimal solution found.")


In [6]:
def get_f0(data:np.array):
    n = data.shape[0]
    m = gp.Model('f0')
    
    # add variables
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')

    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m

In [7]:
def get_f2(data:np.array):
    n = data.shape[0]
    # similar to f0
    m = gp.Model('f2')
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')
    # new constrains for f2
    u = m.addVars(n, vtype=GRB.INTEGER, lb=1, name='u')
    r = r2
    
    m.addConstrs((u[i]-u[j]+1<=n*(1-x[i,j]) for i in range(n) for j in range(n) if i != r and j != r), name='3a')
    m.addConstr(u[r] == 1, name='3b')
    m.addConstrs((u[i]<=n for i in range(n) if i != r), name='3c1')
    m.addConstrs((u[i]>=2 for i in range(n) if i != r), name='3c2')  # added lower bound when defining 'u', do I need this now?
    
    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m


def run_f2(logfileName, params, datasets):
    for name,data in datasets.items():
        with gp.Env() as env:       
        # with gp.Env(f'logs/{logfileName}_{name}.log') as env:       
            with gp.Model(env=env) as model2:
                try:
                    m2 = get_f2(data)
                    m2.setParam("LogFile", f'logs/{logfileName}_{name}.log')
                    for key, value in params.items():
                        m2.setParam(key, value)
                    m2.optimize()
                    print_results(m2)
                except gp.GurobiError as e:
                    print('Error code ' + str(e.errno) + ': ' + str(e))
                except AttributeError:
                    print('Encountered an attribute error')

In [8]:
def get_f3(data:np.array):
    n = data.shape[0]
    # similar to f0
    m = gp.Model('f2')
    x = m.addVars(n, n, vtype=GRB.BINARY, name='x')  # 1e
    m.addConstrs((gp.quicksum(x[i,j] for j in range(n))==1 for i in range(n)),
                 name='1b') 
    m.addConstrs((gp.quicksum(x[i,j] for i in range(n))==1 for j in range(n)),
                 name='1c')
    m.addConstrs((x[i, i] == 0 for i in range(n)),
                 name='1d')
    # f3 constraints
    f = m.addVars(n, n, n, vtype=GRB.BINARY, name='u')
    r = r3
    
    m.addConstrs((gp.quicksum(f[r,j,v] for j in range(n) if j != r) - 
                  gp.quicksum(f[j,r,v] for j in range(n) if j != r) == 1  
                  for v in range(n) 
                  if v != r),
                 name='4a')
    
    m.addConstrs((gp.quicksum(f[i,j,v] for j in range(n) if j != i) - 
                  gp.quicksum(f[j,i,v] for j in range(n) if j != i) == 0  
                  for v in range(n) for i in range(n) 
                  if v != r and i != v), 
                 name='4b')
    
    m.addConstrs((f[i,j,v] <= x[i,j]   
                  for v in range(n) for i in range(n) for j in range(n) 
                  if v != r), 
                 name='4c')
    
    m.setObjective(gp.quicksum(data[i,j]*x[i,j] 
                               for i in range(n) 
                               for j in range(n)),
                   GRB.MINIMIZE)
    return m


def run_f3(logfileName, params, datasets):
    for name,data in datasets.items():
        with gp.Env() as env:       
        # with gp.Env(f'logs/{logfileName}_{name}.log') as env:       
            with gp.Model(env=env) as model2:
                try:
                    m3 = get_f2(data)
                    m3.setParam("LogFile", f'logs/{logfileName}_{name}.log')
                    
                    for key, value in params.items():
                        m3.setParam(key, value)
                    m3.optimize()
                    print_results(m3)
                except gp.GurobiError as e:
                    print('Error code ' + str(e.errno) + ': ' + str(e))
                except AttributeError:
                    print('Encountered an attribute error')

# Testing formulations



In [9]:
# reading datasets first
datasets = read_data('data')

### Test formulation 0 on four instances

In [10]:
for name, data in datasets.items():
    with gp.Env() as env:
    # with gp.Env(f'logs/f0_{name}.log') as env:
        with gp.Model(env=env) as model0:
            try:
                m0 = get_f0(data)
                m0.setParam("LogFile", f'logs/f0_{name}.log')
                m0.optimize()
                print_results(m0)
            except gp.GurobiError as e:
                print('Error code ' + str(e.errno) + ': ' + str(e))
            except AttributeError:
                print('Encountered an attribute error')

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/f0_bays29.log"
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 87 rows, 841 columns and 1711 nonzeros
Model fingerprint: 0x4cfafabf
Variable types: 0 continuous, 841 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6280.0000000
Presolve removed 29 rows and 29 columns
Presolve time: 0.00s
Presolved: 58 rows, 812 c

Thread count was 20 (of 20 available processors)

Solution count 2: 1089 8467 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.089000000000e+03, best bound 1.089000000000e+03, gap 0.0000%
Optimal Solution:
x[0,1]: 1.0
x[1,9]: 1.0
x[2,3]: 1.0
x[3,2]: 1.0
x[4,13]: 1.0
x[5,6]: 1.0
x[6,5]: 1.0
x[7,8]: 1.0
x[8,7]: 1.0
x[9,0]: 1.0
x[10,11]: 1.0
x[11,12]: 1.0
x[12,10]: 1.0
x[13,4]: 1.0
x[14,15]: 1.0
x[15,14]: 1.0
x[16,17]: 1.0
x[17,16]: 1.0
x[18,19]: 1.0
x[19,18]: 1.0
x[20,21]: 1.0
x[21,20]: 1.0
x[22,23]: 1.0
x[23,22]: 1.0
x[24,25]: 1.0
x[25,24]: 1.0
x[26,34]: 1.0
x[27,28]: 1.0
x[28,36]: 1.0
x[29,30]: 1.0
x[30,29]: 1.0
x[31,40]: 1.0
x[32,41]: 1.0
x[33,42]: 1.0
x[34,26]: 1.0
x[35,43]: 1.0
x[36,27]: 1.0
x[37,47]: 1.0
x[38,39]: 1.0
x[39,38]: 1.0
x[40,31]: 1.0
x[41,32]: 1.0
x[42,33]: 1.0
x[43,35]: 1.0
x[44,53]: 1.0
x[45,46]: 1.0
x[46,45]: 1.0
x[47,37]: 1.0
x[48,59]: 1.0
x[49,48]: 1.0
x[50,49]: 1.0
x[51,50]: 1.0
x[52,51]: 1.0
x[53,44]: 1.0
x[54,63]: 1.0
x[55,56]: 1.0
x[56,55]: 1.0
x[

### Test formulation 2 and 3 on four instances



#### a) with default settings: 

In [11]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600}
run_f2('3a_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.01s
Presolved: 814 rows, 840 columns, 3892 nonzeros
Variable types: 0 continuous, 84

     0     0  636.50000    0   87  751.00000  636.50000  15.2%     -    0s
H    0     0                     718.0000000  636.50000  11.4%     -    0s
     0     0  637.25000    0   96  718.00000  637.25000  11.2%     -    0s
     0     0  637.25000    0   69  718.00000  637.25000  11.2%     -    0s
     0     0  637.25000    0   96  718.00000  637.25000  11.2%     -    0s
     0     0  637.25000    0  107  718.00000  637.25000  11.2%     -    0s
     0     0  638.00000    0  119  718.00000  638.00000  11.1%     -    0s
     0     0  638.00000    0   68  718.00000  638.00000  11.1%     -    0s
     0     2  638.50000    0   62  718.00000  638.50000  11.1%     -    0s
H   30    39                     715.0000000  638.50000  10.7%  33.3    0s
H   95   106                     710.0000000  638.50000  10.1%  20.0    0s
*  647   691              47     707.0000000  638.50000  9.69%   8.3    0s
H 2412  1762                     704.0000000  682.91403  3.00%  10.6    2s
H 2413  1675             

     0     0 104100.000    0  217 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   70 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   74 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   74 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   73 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   69 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   67 110340.000 104100.000  5.66%     -    2s
     0     2 104338.000    0   64 110340.000 104338.000  5.44%     -    2s
H  113   124                    110261.00000 104338.000  5.37%  43.7    2s
H  259   262                    110135.00000 104338.000  5.26%  35.2    3s
H  260   262                    108736.00000 104338.000  4.04%  35.2    3s
  2980  1613 105839.530   18  196 108736.000 104712.250  3.70%  22.3    5s
H 2982  1533                    108559.00000 104712.250  3.54%  22.3    5s
H 3005  1471             

H    0     0                    1365.0000000 1199.50000  12.1%     -    1s
H    0     0                    1333.0000000 1199.50000  10.0%     -    1s
     0     0 1199.50000    0  260 1333.00000 1199.50000  10.0%     -    1s
     0     0 1199.50000    0  232 1333.00000 1199.50000  10.0%     -    1s
H    0     0                    1309.0000000 1199.50000  8.37%     -    1s
     0     0 1199.50000    0  231 1309.00000 1199.50000  8.37%     -    1s
     0     0 1199.50000    0  161 1309.00000 1199.50000  8.37%     -    1s
H    0     0                    1278.0000000 1199.50000  6.14%     -    1s
     0     0 1199.50000    0  240 1278.00000 1199.50000  6.14%     -    1s
     0     0 1199.50000    0  177 1278.00000 1199.50000  6.14%     -    2s
     0     0 1199.50000    0  246 1278.00000 1199.50000  6.14%     -    2s
     0     0 1199.50000    0  248 1278.00000 1199.50000  6.14%     -    2s
     0     0 1199.50000    0  240 1278.00000 1199.50000  6.14%     -    2s
H    0     0             

In [12]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600}
run_f3('3a_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3a_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.01s
Presolved: 814 rows, 840 columns, 3892 nonzeros
Variable types: 0 continuous, 84

     0     0  636.50000    0   87  751.00000  636.50000  15.2%     -    0s
H    0     0                     718.0000000  636.50000  11.4%     -    0s
     0     0  637.25000    0   96  718.00000  637.25000  11.2%     -    0s
     0     0  637.25000    0   69  718.00000  637.25000  11.2%     -    0s
     0     0  637.25000    0   96  718.00000  637.25000  11.2%     -    0s
     0     0  637.25000    0  107  718.00000  637.25000  11.2%     -    0s
     0     0  638.00000    0  119  718.00000  638.00000  11.1%     -    0s
     0     0  638.00000    0   68  718.00000  638.00000  11.1%     -    0s
     0     2  638.50000    0   62  718.00000  638.50000  11.1%     -    0s
H   30    39                     715.0000000  638.50000  10.7%  33.3    0s
H   95   106                     710.0000000  638.50000  10.1%  20.0    0s
*  647   691              47     707.0000000  638.50000  9.69%   8.3    0s
H 2412  1762                     704.0000000  682.91403  3.00%  10.6    2s
H 2413  1675             

     0     0 104100.000    0  217 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   70 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   74 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   74 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   73 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   69 110340.000 104100.000  5.66%     -    2s
     0     0 104100.000    0   67 110340.000 104100.000  5.66%     -    2s
     0     2 104338.000    0   64 110340.000 104338.000  5.44%     -    2s
H  113   124                    110261.00000 104338.000  5.37%  43.7    3s
H  259   262                    110135.00000 104338.000  5.26%  35.2    3s
H  260   262                    108736.00000 104338.000  4.04%  35.2    3s
  2963  1602 107515.000   24  158 108736.000 104712.250  3.70%  22.4    5s
H 2982  1533                    108559.00000 104712.250  3.54%  22.3    6s
H 3005  1471             

H    0     0                    1382.0000000 1199.50000  13.2%     -    1s
     0     0 1199.50000    0  240 1382.00000 1199.50000  13.2%     -    1s
H    0     0                    1365.0000000 1199.50000  12.1%     -    1s
H    0     0                    1333.0000000 1199.50000  10.0%     -    1s
     0     0 1199.50000    0  260 1333.00000 1199.50000  10.0%     -    1s
     0     0 1199.50000    0  232 1333.00000 1199.50000  10.0%     -    1s
H    0     0                    1309.0000000 1199.50000  8.37%     -    1s
     0     0 1199.50000    0  231 1309.00000 1199.50000  8.37%     -    1s
     0     0 1199.50000    0  161 1309.00000 1199.50000  8.37%     -    1s
H    0     0                    1278.0000000 1199.50000  6.14%     -    2s
     0     0 1199.50000    0  240 1278.00000 1199.50000  6.14%     -    2s
     0     0 1199.50000    0  177 1278.00000 1199.50000  6.14%     -    2s
     0     0 1199.50000    0  246 1278.00000 1199.50000  6.14%     -    2s
     0     0 1199.50000  

#### b) with `presolve` truned off: 

In [13]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0}
run_f2('3b_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3b_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Presolve  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Variable types: 0 continuous, 870 integer (841 binary)

Root relaxation: objective 1.774000e+03,

H    0     0                     722.0000000  638.00000  11.6%     -    0s
     0     2  638.00000    0   57  722.00000  638.00000  11.6%     -    0s
H 2080  1645                     721.0000000  657.58408  8.80%  10.9    3s
H 2084  1565                     719.0000000  681.13924  5.27%  10.9    3s
H 2121  1526                     712.0000000  682.28433  4.17%  12.5    3s
* 2646  1540              29     709.0000000  683.04762  3.66%  14.2    4s
H 3095  1442                     708.0000000  685.41032  3.19%  14.7    4s
H 3106  1404                     699.0000000  685.41032  1.94%  14.8    4s
  4116   788     cutoff   18       699.00000  690.21482  1.26%  15.9    5s

Cutting planes:
  Learned: 61
  Gomory: 48
  Lift-and-project: 22
  Cover: 4
  Implied bound: 13
  Projected implied bound: 24
  Clique: 2
  MIR: 49
  Mixing: 11
  StrongCG: 6
  Flow cover: 179
  Inf proof: 7
  Zero half: 30
  Mod-K: 4
  RLT: 13
  Relax-and-lift: 34
  BQP: 1
  PSD: 6

Explored 4477 nodes (69881 simplex ite

H 5530  1812                    112975.00000 104742.968  7.29%  12.0   11s
H 5599  1775                    112961.00000 104880.000  7.15%  13.0   12s
H 5900  1871                    112882.00000 104880.000  7.09%  13.9   12s
  7064  2314 108296.539   58  159 112882.000 104880.000  7.09%  15.9   15s
H 7906  2453                    112621.00000 104903.351  6.85%  16.8   15s
H 8813  2672                    112588.00000 104931.447  6.80%  17.6   17s
H 8833  2522                    112157.00000 104931.447  6.44%  17.6   17s
H 9054  2538                    112124.00000 104931.447  6.41%  17.7   17s
H 9635  2328                    110761.00000 105007.605  5.19%  17.9   18s
 10236  2902 105715.234   20  125 110761.000 105090.257  5.12%  18.5   20s
 15438  6503 109745.142   38  108 110761.000 105463.880  4.78%  20.0   25s
H18272  7548                    110517.00000 105580.904  4.47%  20.6   28s
H18282  7177                    110438.00000 105580.904  4.40%  20.5   29s
H18283  6819             

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1090.89899    0  173          - 1090.89899      -     -    0s
     0     0 1185.71212    0  258          - 1185.71212      -     -    0s
     0     0 1185.71212    0  240          - 1185.71212      -     -    0s
     0     0 1199.01010    0  254          - 1199.01010      -     -    0s
     0     0 1199.01010    0  260          - 1199.01010      -     -    0s
     0     0 1199.50000    0  225          - 1199.50000      -     -    0s
     0     0 1199.50000    0  230          - 1199.50000      -     -    0s
H    0     0                    2204.0000000 1199.50000  45.6%     -    0s
H    0     0                    2015.0000000 1199.50000  40.5%     -    1s
H    0     0                    1911.0000000 1199.50000  37.2%     -    1s
H    0     0                    1829.0000000 1199.50000  34.4%     -    1s
H    0     0                    1811.0000000 1199.50000  33.8%     -    1s
H    0     0          

In [14]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0}
run_f3('3b_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3b_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Presolve  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Variable types: 0 continuous, 870 integer (841 binary)

Root relaxation: objective 1.774000e+03,

H    0     0                     722.0000000  638.00000  11.6%     -    0s
     0     2  638.00000    0   57  722.00000  638.00000  11.6%     -    0s
H 2080  1645                     721.0000000  657.58408  8.80%  10.9    3s
H 2084  1565                     719.0000000  681.13924  5.27%  10.9    3s
H 2121  1526                     712.0000000  682.28433  4.17%  12.5    3s
* 2646  1540              29     709.0000000  683.04762  3.66%  14.2    4s
H 3095  1442                     708.0000000  685.41032  3.19%  14.7    4s
H 3106  1404                     699.0000000  685.41032  1.94%  14.8    4s
  3963   929     cutoff   20       699.00000  689.58126  1.35%  15.8    5s

Cutting planes:
  Learned: 61
  Gomory: 48
  Lift-and-project: 22
  Cover: 4
  Implied bound: 13
  Projected implied bound: 24
  Clique: 2
  MIR: 49
  Mixing: 11
  StrongCG: 6
  Flow cover: 179
  Inf proof: 7
  Zero half: 30
  Mod-K: 4
  RLT: 13
  Relax-and-lift: 34
  BQP: 1
  PSD: 6

Explored 4477 nodes (69881 simplex ite

H 5530  1812                    112975.00000 104742.968  7.29%  12.0   11s
H 5599  1775                    112961.00000 104880.000  7.15%  13.0   12s
H 5900  1871                    112882.00000 104880.000  7.09%  13.9   12s
  7064  2314 108296.539   58  159 112882.000 104880.000  7.09%  15.9   15s
H 7906  2453                    112621.00000 104903.351  6.85%  16.8   16s
H 8813  2672                    112588.00000 104931.447  6.80%  17.6   17s
H 8833  2522                    112157.00000 104931.447  6.44%  17.6   17s
H 9054  2538                    112124.00000 104931.447  6.41%  17.7   17s
H 9635  2328                    110761.00000 105007.605  5.19%  17.9   19s
 10236  2902 105715.234   20  125 110761.000 105090.257  5.12%  18.5   20s
 14876  6146 109962.667   28   37 110761.000 105411.184  4.83%  19.9   25s
H18272  7548                    110517.00000 105580.904  4.47%  20.6   28s
H18282  7177                    110438.00000 105580.904  4.40%  20.5   29s
H18283  6819             

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1090.89899    0  173          - 1090.89899      -     -    0s
     0     0 1185.71212    0  258          - 1185.71212      -     -    0s
     0     0 1185.71212    0  240          - 1185.71212      -     -    0s
     0     0 1199.01010    0  254          - 1199.01010      -     -    0s
     0     0 1199.01010    0  260          - 1199.01010      -     -    0s
     0     0 1199.50000    0  225          - 1199.50000      -     -    0s
     0     0 1199.50000    0  230          - 1199.50000      -     -    0s
H    0     0                    2204.0000000 1199.50000  45.6%     -    1s
H    0     0                    2015.0000000 1199.50000  40.5%     -    1s
H    0     0                    1911.0000000 1199.50000  37.2%     -    1s
H    0     0                    1829.0000000 1199.50000  34.4%     -    1s
H    0     0                    1811.0000000 1199.50000  33.8%     -    1s
H    0     0          

#### c) with `cuts` turned off: 

In [15]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Cuts':0}
run_f2('3c_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Cuts  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.01s
Presolved: 814 rows, 840 columns, 3892 non

Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f2_pr76.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Cuts  0

Optimize a model with 6004 rows, 5852 columns and 28504 nonzeros
Model fingerprint: 0x3e6cdb56
Variable types: 0 continuous, 5852 integer (5776 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+01]
  Objective range  [3e+02, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 302 rows and 77 columns
Presolve time: 0.03s
Presolved: 5702 rows, 5775 columns, 28050 nonzeros
Variable ty

 374414 178840 99332.5820   29   97 108159.000 95525.3196  11.7%  19.8  295s
 377913 180571     cutoff   37      108159.000 95542.4553  11.7%  19.8  300s
 382939 182817 104510.083   31  105 108159.000 95571.0858  11.6%  19.8  305s
 386818 184737 infeasible   43      108159.000 95591.8773  11.6%  19.8  310s
 391896 186935 infeasible   36      108159.000 95612.7793  11.6%  19.8  316s
 395743 188306 107231.062   33  104 108159.000 95629.2679  11.6%  19.8  320s
 399671 189766 infeasible   45      108159.000 95654.7512  11.6%  19.8  325s
 404595 191863 107027.892   37   91 108159.000 95675.8998  11.5%  19.8  331s
 408268 193522 96892.2011   26  135 108159.000 95696.3232  11.5%  19.8  335s
 413402 195629 100818.796   32  103 108159.000 95720.4506  11.5%  19.8  341s
 417039 197035 97641.3220   25   98 108159.000 95735.1077  11.5%  19.8  345s
 421128 198620 97078.0627   26   87 108159.000 95755.7206  11.5%  19.8  350s
 423009 199718 101031.976   30  135 108159.000 95761.7202  11.5%  19.8  355s

 713686 322434 107841.244   36   92 108159.000 96609.4831  10.7%  19.7  831s
 716356 323421 100317.759   29  109 108159.000 96616.0106  10.7%  19.7  836s
 718934 324742 98551.3607   27  131 108159.000 96621.9629  10.7%  19.7  840s
 721450 325741 99927.9898   29  149 108159.000 96627.6518  10.7%  19.7  845s
 725245 327440 101192.430   33  100 108159.000 96633.0395  10.7%  19.7  851s
 727645 328458 106877.909   40   95 108159.000 96636.0227  10.7%  19.7  856s
 730184 329445 101003.578   26   95 108159.000 96641.1939  10.6%  19.7  861s
 732526 330586 infeasible   38      108159.000 96645.9807  10.6%  19.7  865s
 735058 331650 106891.760   40  116 108159.000 96651.5885  10.6%  19.7  870s
 738981 333510 106182.967   36  113 108159.000 96660.8728  10.6%  19.7  876s
 741566 334623     cutoff   43      108159.000 96666.9795  10.6%  19.7  882s
 744032 335705 105436.845   36  111 108159.000 96673.0224  10.6%  19.7  886s
 746870 336802 104924.581   35  112 108159.000 96676.5871  10.6%  19.7  891s

 970465 428917 99589.0269   29  151 108159.000 97098.8441  10.2%  19.6 1367s
 972321 429450 107330.214   45  110 108159.000 97100.9966  10.2%  19.6 1371s
 975116 430623 99025.5493   27  139 108159.000 97106.2444  10.2%  19.6 1377s
 976642 431175 106434.971   43  105 108159.000 97107.8386  10.2%  19.6 1380s
 979437 432334 infeasible   37      108159.000 97113.1051  10.2%  19.6 1386s
 980959 432833 103815.263   33   99 108159.000 97116.2493  10.2%  19.6 1390s
 983700 433898 106969.623   38  107 108159.000 97121.6486  10.2%  19.6 1396s
 986277 434378 97162.0018   25  120 108159.000 97125.6223  10.2%  19.6 1401s
 988084 435437 104329.664   33  115 108159.000 97128.3245  10.2%  19.6 1408s
 989336 436199 97802.5168   27  114 108159.000 97131.0462  10.2%  19.6 1411s
 992029 437516 infeasible   36      108159.000 97134.6344  10.2%  19.6 1417s
 993483 437968 97214.6189   26  132 108159.000 97136.9078  10.2%  19.6 1421s
 996213 439028     cutoff   39      108159.000 97141.8663  10.2%  19.6 1427s

...

 130621 46210 1177.62337   28  215 1218.00000 1165.75258  4.29%  22.1  220s
 135746 47541 1207.49485   36  128 1218.00000 1166.12371  4.26%  22.3  225s
 138846 48548     cutoff   37      1218.00000 1166.68076  4.21%  22.3  231s
 142670 49805 1214.34536   35  127 1218.00000 1166.98969  4.19%  22.4  235s
 148249 51650 1208.86318   35  324 1218.00000 1167.69966  4.13%  22.5  241s
 152072 53373 1182.74708   30  160 1218.00000 1167.90756  4.11%  22.6  246s
 157186 54595 1202.08522   37  168 1218.00000 1168.14433  4.09%  22.7  251s
 160229 55455 1185.51753   28  129 1218.00000 1168.64811  4.05%  22.7  255s
 161608 55938 1194.90292   34  226 1218.00000 1168.68557  4.05%  22.7  260s
 163464 57056 1213.15464   39  176 1218.00000 1168.76976  4.04%  22.7  266s
 166552 57936 infeasible   35      1218.00000 1168.93814  4.03%  22.8  270s
 170438 59036     cutoff   35      1218.00000 1169.20619  4.01%  22.8  275s
 174313 60446 1214.17732   36  125 1218.00000 1169.73196  3.96%  22.9  280s
 178860 6195

 485105 112783     cutoff   36      1211.00000 1181.91055  2.40%  23.7  756s
 487563 112985     cutoff   35      1211.00000 1181.95533  2.40%  23.7  762s
 490248 113237 1204.62887   35   89 1211.00000 1182.02062  2.39%  23.7  767s
 493004 113490 1197.44330   27  184 1211.00000 1182.09794  2.39%  23.7  772s
 495667 113583 1202.22371   41  164 1211.00000 1182.22062  2.38%  23.7  778s
 496719 113756 1182.61856   30  131 1211.00000 1182.28041  2.37%  23.7  786s
 498300 113967 1194.06186   32  131 1211.00000 1182.35395  2.37%  23.7  791s
 499192 114190 infeasible   30      1211.00000 1182.39863  2.36%  23.7  795s
 500228 114421 infeasible   38      1211.00000 1182.43299  2.36%  23.7  800s
 501208 114675 1191.54124   31  128 1211.00000 1182.45601  2.36%  23.7  805s
 502368 115070 1202.54639   34  248 1211.00000 1182.48247  2.35%  23.7  812s
 504074 115252 1205.64261   32  123 1211.00000 1182.52234  2.35%  23.7  818s
 505090 115528 1190.54639   30  224 1211.00000 1182.53608  2.35%  23.7  824s

 623418 129750 1204.51497   36  148 1211.00000 1184.68041  2.17%  23.6 1429s
 624331 129863 1196.56014   34  196 1211.00000 1184.68299  2.17%  23.6 1435s
 625730 129984 1192.76632   33  162 1211.00000 1184.70361  2.17%  23.6 1443s
 626832 130132 1196.94502   34  167 1211.00000 1184.71478  2.17%  23.6 1452s
 628676 130282 1200.50945   32  272 1211.00000 1184.72852  2.17%  23.5 1457s
 629708 130384 1198.71005   39  294 1211.00000 1184.74227  2.17%  23.5 1463s
 630815 130464 1198.75601   34  159 1211.00000 1184.75258  2.17%  23.6 1471s
 632386 130534 1198.93814   32  109 1211.00000 1184.76804  2.17%  23.6 1477s
 633235 130630 1198.49943   32  184 1211.00000 1184.77835  2.17%  23.6 1484s
 634316 130732     cutoff   36      1211.00000 1184.79381  2.16%  23.6 1490s
 635418 130880 1205.60825   32  163 1211.00000 1184.80412  2.16%  23.6 1497s
 637005 130962 1187.97113   36  209 1211.00000 1184.82474  2.16%  23.6 1503s
 638087 131121 1187.82852   35  123 1211.00000 1184.84192  2.16%  23.6 1512s

 766877 144132 infeasible   35      1211.00000 1186.71134  2.01%  23.4 2311s
 768739 144287 1201.62405   34  200 1211.00000 1186.72302  2.00%  23.4 2322s
 770715 144432 infeasible   36      1211.00000 1186.74227  2.00%  23.4 2332s
 772481 144493     cutoff   31      1211.00000 1186.75773  2.00%  23.4 2341s
 774087 144579     cutoff   35      1211.00000 1186.77320  2.00%  23.4 2348s
 775336 144629 1204.75515   36  186 1211.00000 1186.78557  2.00%  23.4 2356s
 776398 144688 1194.75876   35  211 1211.00000 1186.79931  2.00%  23.4 2361s
 777374 144729 1206.45017   35  165 1211.00000 1186.80756  2.00%  23.4 2367s
 778038 144823 1205.53780   31  121 1211.00000 1186.81443  2.00%  23.4 2373s
 779343 144874 infeasible   38      1211.00000 1186.82543  2.00%  23.4 2381s
 780429 144955 infeasible   37      1211.00000 1186.84227  1.99%  23.4 2389s
 781994 144955 1195.03093   34  151 1211.00000 1186.86082  1.99%  23.4 2395s
 783041 144958 1202.77320   36  112 1211.00000 1186.87285  1.99%  23.4 2402s

 897038 154733 infeasible   34      1211.00000 1188.49691  1.86%  23.3 3113s
 897790 154862 1201.06186   39  145 1211.00000 1188.50172  1.86%  23.3 3119s
 898814 155088 1205.22680   44  147 1211.00000 1188.51031  1.86%  23.3 3127s
 900365 155177 1205.90034   34  157 1211.00000 1188.52062  1.86%  23.3 3133s
 901397 155289 infeasible   39      1211.00000 1188.52921  1.86%  23.3 3141s
 902775 155315 infeasible   29      1211.00000 1188.53952  1.85%  23.3 3147s
 903843 155441 infeasible   32      1211.00000 1188.54570  1.85%  23.3 3155s
 904945 155651 1205.34536   32  149 1211.00000 1188.55155  1.85%  23.3 3164s
 906643 155755 1201.76289   28  115 1211.00000 1188.56357  1.85%  23.3 3171s
 907718 155877     cutoff   33      1211.00000 1188.57290  1.85%  23.3 3179s
 909304 155964 1188.62887   35   97 1211.00000 1188.58282  1.85%  23.3 3185s
 910326 156009 1204.38488   36   80 1211.00000 1188.59107  1.85%  23.3 3192s
 911464 156130 1188.70997   29  259 1211.00000 1188.60000  1.85%  23.3 3202s

In [16]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Cuts':0}
run_f3('3c_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3c_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Cuts to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Cuts  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 114 rows and 30 columns
Presolve time: 0.01s
Presolved: 814 rows, 840 columns, 3892 non

 108184 15497 infeasible   35       699.00000  653.10000  6.57%  10.3  111s
 113012 15404     cutoff   31       699.00000  653.85000  6.46%  10.3  115s
 117593 15070  656.86667   27   57  699.00000  654.66875  6.34%  10.2  121s
 122427 14687  672.48333   24   53  699.00000  655.50000  6.22%  10.2  126s
 127145 14397 infeasible   37       699.00000  656.30000  6.11%  10.2  131s
 131548 14060  658.95833   28   43  699.00000  657.20000  5.98%  10.1  136s
 136236 13640     cutoff   30       699.00000  658.28333  5.82%  10.1  142s
 140601 13140     cutoff   25       699.00000  659.23333  5.69%  10.1  147s
 142860 12870  683.94167   29   61  699.00000  660.01667  5.58%  10.1  150s
 147502 12229     cutoff   27       699.00000  661.31250  5.39%  10.0  155s
 151990 11458  668.00000   30   45  699.00000  662.61667  5.21%  10.0  161s
 156187 10469 infeasible   43       699.00000  664.17500  4.98%  10.0  165s
 162305  8216 infeasible   26       699.00000  668.70000  4.33%  10.0  171s
 166256  682

 40879 22964 96661.2899   28  106 109006.000 91033.4993  16.5%  19.5  137s
 42022 23575 104041.762   32   90 109006.000 91142.2151  16.4%  19.6  140s
 44086 25000 96096.4899   25  114 109006.000 91399.5829  16.2%  19.6  147s
 45716 25559 94633.6791   23  113 109006.000 91483.8699  16.1%  19.6  150s
 47854 26788 93168.9222   21  119 109006.000 91670.9584  15.9%  19.7  157s
 49156 27569 94178.6047   22  104 109006.000 91782.8681  15.8%  19.7  160s
 51290 29007 97690.0259   30   97 109006.000 91920.9995  15.7%  19.7  166s
 54256 30359 92093.0173   21  131 109006.000 91994.5736  15.6%  19.7  174s
 55727 31308 99384.5213   28  106 109006.000 92026.8196  15.6%  19.7  177s
 57451 31813 104610.673   39  110 109006.000 92080.5632  15.5%  19.7  180s
 59271 33134 infeasible   39      109006.000 92126.9639  15.5%  19.7  186s
 62502 34466 100735.344   33   89 109006.000 92177.6668  15.4%  19.6  191s
 64813 35617 94640.9123   25  114 109006.000 92224.6249  15.4%  19.6  196s
 67161 36671 99390.7781  

 291803 142815 107216.085   37   71 108264.000 95056.0108  12.2%  19.8  662s
 292954 143496 101907.220   33  118 108264.000 95065.5329  12.2%  19.8  665s
 295610 144994 96487.2840   26  103 108264.000 95075.8638  12.2%  19.8  671s
 298228 146361 98980.6163   25  126 108264.000 95090.0880  12.2%  19.8  677s
 299609 146799 98742.0622   26  108 108264.000 95097.0303  12.2%  19.8  680s
 302009 148012 104452.937   32  110 108264.000 95111.5034  12.1%  19.8  686s
 304219 149174 infeasible   36      108264.000 95123.6887  12.1%  19.8  692s
 305564 149813 107202.027   34  115 108264.000 95128.4658  12.1%  19.8  696s
 308275 151010 107456.160   44   93 108264.000 95146.1054  12.1%  19.8  702s
 309339 151637 106888.507   37  119 108264.000 95151.5579  12.1%  19.8  705s
 312089 152989 102060.717   28  120 108264.000 95167.9695  12.1%  19.8  713s
 313430 153474 infeasible   37      108264.000 95176.0124  12.1%  19.8  716s
 316038 154729 106216.977   32   87 108264.000 95196.0022  12.1%  19.8  723s

 463352 216659 102310.170   31  102 108159.000 95923.8323  11.3%  19.8 1208s
 464685 217048 infeasible   37      108159.000 95927.8039  11.3%  19.8 1213s
 465896 217750 96018.2971   24  140 108159.000 95931.9634  11.3%  19.8 1218s
 467139 218169 106543.390   35  119 108159.000 95934.6193  11.3%  19.8 1222s
 468434 218744     cutoff   41      108159.000 95937.3886  11.3%  19.8 1227s
 469519 219410 99053.3570   29  125 108159.000 95940.3468  11.3%  19.8 1233s
 470968 219908 107141.271   44  127 108159.000 95945.9097  11.3%  19.8 1238s
 472345 220394 106925.207   35   88 108159.000 95948.7467  11.3%  19.8 1243s
 473674 220793 104733.795   33   90 108159.000 95951.8112  11.3%  19.8 1247s
 474838 221319 105795.727   41  115 108159.000 95959.0277  11.3%  19.8 1253s
 476107 221727 100445.151   28  123 108159.000 95962.1160  11.3%  19.8 1259s
 477636 222318 98920.1183   27  120 108159.000 95966.5341  11.3%  19.8 1264s
 478831 222791 100850.053   30  111 108159.000 95972.6818  11.3%  19.8 1268s

...

 108375 38556 infeasible   43      1218.00000 1163.00550  4.52%  21.5  529s
 109847 38830     cutoff   32      1218.00000 1163.18763  4.50%  21.6  533s
 110950 39041 1200.10567   38  276 1218.00000 1163.38557  4.48%  21.6  538s
 112133 39542 1196.52577   35   91 1218.00000 1163.65979  4.46%  21.7  542s
 113218 40112 1194.61426   29  137 1218.00000 1163.76289  4.45%  21.7  547s
 114830 40562 infeasible   35      1218.00000 1163.87457  4.44%  21.7  552s
 116108 40926 1210.46544   33  255 1218.00000 1163.95876  4.44%  21.7  556s
 118440 41558 1190.68557   32  212 1218.00000 1164.17526  4.42%  21.8  563s
 119666 42143 1211.89175   43  187 1218.00000 1164.35258  4.40%  21.8  568s
 120995 42470 1183.19914   28  185 1218.00000 1164.72165  4.37%  21.9  571s
 122214 43004 1169.97079   28  176 1218.00000 1164.83505  4.36%  21.9  575s
 123388 43495 1209.76289   37  197 1218.00000 1164.90722  4.36%  21.9  580s
 125839 44317 1180.81959   29  216 1218.00000 1165.05052  4.35%  22.0  588s
 127040 4468

 254185 84342 1211.65945   35  232 1218.00000 1173.93814  3.62%  23.2 1092s
 255161 84705 1208.56014   37  172 1218.00000 1173.95876  3.62%  23.2 1097s
 256669 84993 infeasible   32      1218.00000 1174.00515  3.61%  23.2 1101s
 257771 85268 infeasible   37      1218.00000 1174.06048  3.61%  23.2 1107s
 259085 85495 1175.77320   26  124 1218.00000 1174.11890  3.60%  23.3 1112s
 260238 85808 1179.93729   30  207 1218.00000 1174.18557  3.60%  23.3 1117s
 261492 86237 1213.89863   32  249 1218.00000 1174.29897  3.59%  23.3 1122s
 263239 86535 infeasible   36      1218.00000 1174.52921  3.57%  23.3 1126s
 264314 87182     cutoff   38      1218.00000 1174.60052  3.56%  23.3 1131s
 266861 88025 1203.76289   31  122 1218.00000 1174.69543  3.56%  23.3 1140s
 269315 88560 1206.05241   47  113 1218.00000 1174.73883  3.55%  23.3 1145s
 270816 89366 1205.67784   33  115 1218.00000 1174.79072  3.55%  23.3 1153s
 272339 89765 1180.71478   35  213 1218.00000 1174.82589  3.54%  23.3 1157s
 273466 9020

 399929 101395     cutoff   35      1211.00000 1179.78855  2.58%  23.6 1670s
 401067 101645 1199.94845   35  147 1211.00000 1179.80481  2.58%  23.6 1676s
 402629 101820 1193.93849   29  224 1211.00000 1179.83505  2.57%  23.6 1681s
 403755 101957     cutoff   36      1211.00000 1179.84880  2.57%  23.6 1686s
 404927 102083 infeasible   30      1211.00000 1179.87973  2.57%  23.6 1692s
 405998 102157 1197.66323   34  175 1211.00000 1179.89897  2.57%  23.6 1698s
 406807 102216 1206.18454   32  116 1211.00000 1179.90722  2.57%  23.6 1707s
 407705 102335 1188.50515   32  133 1211.00000 1179.91237  2.57%  23.6 1715s
 408863 102448 infeasible   33      1211.00000 1179.91468  2.57%  23.6 1724s
 410515 102515 1193.64948   37  143 1211.00000 1180.01753  2.56%  23.6 1733s
 412090 102724 1199.84536   38  134 1211.00000 1180.08333  2.55%  23.6 1741s
 413824 102814 1197.38918   32  140 1211.00000 1180.12371  2.55%  23.6 1748s
 415349 102961 1183.42268   28  112 1211.00000 1180.25017  2.54%  23.7 1755s

 537994 119261 1203.76357   36  322 1211.00000 1183.02663  2.31%  23.6 2329s
 539071 119298 infeasible   34      1211.00000 1183.06186  2.31%  23.6 2335s
 540199 119370 infeasible   35      1211.00000 1183.10570  2.30%  23.6 2341s
 541335 119458 1197.32990   33   48 1211.00000 1183.15120  2.30%  23.7 2348s
 542633 119563 infeasible   35      1211.00000 1183.21649  2.29%  23.7 2354s
 543771 119761 infeasible   37      1211.00000 1183.29897  2.29%  23.7 2360s
 544948 120101 1185.40206   35  111 1211.00000 1183.33471  2.28%  23.7 2368s
 546649 120405     cutoff   39      1211.00000 1183.41237  2.28%  23.6 2375s
 548189 120616 1203.38660   40   82 1211.00000 1183.45361  2.27%  23.6 2381s
 549324 120854 1202.82801   38  261 1211.00000 1183.47938  2.27%  23.6 2387s
 550383 121062 1207.43814   32  131 1211.00000 1183.49227  2.27%  23.6 2394s
 551465 121272 infeasible   38      1211.00000 1183.51546  2.27%  23.6 2401s
 552828 121566 infeasible   37      1211.00000 1183.53608  2.27%  23.6 2408s

 670558 134955 infeasible   46      1211.00000 1185.53265  2.10%  23.5 3088s
 672045 135108 1206.81787   38  172 1211.00000 1185.54845  2.10%  23.5 3094s
 673015 135189     cutoff   36      1211.00000 1185.55670  2.10%  23.5 3100s
 673927 135294 infeasible   38      1211.00000 1185.56701  2.10%  23.5 3106s
 674943 135494 1203.58027   30  174 1211.00000 1185.57388  2.10%  23.5 3114s
 676159 135604 1194.41237   32  110 1211.00000 1185.58282  2.10%  23.5 3121s
 677145 135873 1203.55670   32  186 1211.00000 1185.59107  2.10%  23.5 3129s
 678763 135976     cutoff   38      1211.00000 1185.60273  2.10%  23.5 3136s
 679886 136072     cutoff   37      1211.00000 1185.61168  2.10%  23.5 3143s
 680945 136347 infeasible   36      1211.00000 1185.62062  2.10%  23.5 3153s
 682868 136439 1205.40206   31  171 1211.00000 1185.63780  2.09%  23.5 3160s
 683832 136506 1205.41237   36  162 1211.00000 1185.64948  2.09%  23.5 3167s
 684982 136700 1188.00369   30  272 1211.00000 1185.65155  2.09%  23.5 3176s

#### d) with both `presolve` and `cuts` turned off

In [17]:
# FORMULATION 2
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0, 'Cuts':0}
run_f2('3d_f2', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3d_f2_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Cuts  0
Presolve  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Variable types: 0 continuous, 870 integer (841 binary)

Ro

 142144 54481  652.05691   30   37  699.00000  651.09756  6.85%  10.3  125s
 148128 55808     cutoff   34       699.00000  651.53659  6.79%  10.3  131s
 154523 57021  666.12195   47   35  699.00000  652.15854  6.70%  10.4  136s
 158735 57699  657.45122   25   44  699.00000  652.57317  6.64%  10.5  140s
 165530 59105  655.43570   27   66  699.00000  653.16260  6.56%  10.6  145s
 171610 60316 infeasible   28       699.00000  653.61789  6.49%  10.7  151s
 178414 61424  656.20732   33   61  699.00000  654.26016  6.40%  10.7  156s
 182111 61921  664.97561   36   30  699.00000  654.47967  6.37%  10.8  160s
 188689 63146 infeasible   29       699.00000  655.03049  6.29%  10.9  166s
 195612 64935     cutoff   36       699.00000  655.36585  6.24%  10.8  170s
 201783 66859  696.78049   49   14  699.00000  655.58537  6.21%  10.8  175s
 208326 68813  667.56098   35   54  699.00000  655.82114  6.18%  10.8  180s
 215162 70697  688.25203   29   50  699.00000  656.02439  6.15%  10.7  185s
 223859 7364

     0     0 77656.2237    0  125 201534.000 77656.2237  61.5%     -    1s
H    0     0                    173657.00000 78991.2237  54.5%     -    1s
     0     2 78991.2237    0  125 173657.000 78991.2237  54.5%     -    1s
H   27    50                    173649.00000 78991.2237  54.5%  13.4    2s
   792   867 91304.5658   56   93 173649.000 78991.2237  54.5%  20.4    5s
H 1168  1178                    172350.00000 78991.2237  54.2%  17.0    6s
H 1172  1178                    168662.00000 78991.2237  53.2%  17.0    6s
H 1197  1178                    155513.00000 78991.2237  49.2%  16.8    6s
H 1549  1535                    150420.00000 78991.2237  47.5%  15.4    8s
  2027  2095 96385.0987  149   93 150420.000 78991.2237  47.5%  14.6   10s
H 2582  2564                    150332.00000 78991.2237  47.5%  13.1   12s
H 3043  2974                    149110.00000 78991.2237  47.0%  12.4   13s
H 3104  3086                    147938.00000 78991.2237  46.6%  12.4   14s
H 3143  3086             

 138713 70430 99344.0000   29  123 108234.000 94223.4578  12.9%  17.2  365s
 142580 72407 105471.438   39   95 108234.000 94261.1778  12.9%  17.2  370s
 146311 74537 99249.3000   29  117 108234.000 94292.8867  12.9%  17.3  375s
 147558 75146     cutoff   43      108234.000 94295.4911  12.9%  17.3  380s
 151614 77325 103958.278   30  111 108234.000 94332.1178  12.8%  17.3  386s
 154274 78756 107611.413   43   80 108234.000 94357.9267  12.8%  17.3  391s
 156992 79813 107433.320   39   93 108234.000 94370.7889  12.8%  17.3  395s
 161128 81726 104533.280   36   73 108234.000 94420.6844  12.8%  17.3  401s
 163534 83162 94596.9689   28  108 108234.000 94437.1822  12.7%  17.3  405s
 167646 84818 97908.9200   31   98 108234.000 94469.2267  12.7%  17.3  410s
 170903 86453 106127.067   41   76 108234.000 94493.6511  12.7%  17.4  416s
 173283 87373 107696.080   38   61 108234.000 94512.4400  12.7%  17.4  420s
 175423 88486 107572.947   41  103 108234.000 94532.7111  12.7%  17.4  425s
 177464 8962

 347749 161524 104773.847   37   76 108159.000 95468.9511  11.7%  17.7  896s
 348935 161966 104841.729   32  119 108159.000 95473.3878  11.7%  17.7  900s
 351465 163213 96492.2956   29  106 108159.000 95481.7333  11.7%  17.7  907s
 352720 163772 infeasible   33      108159.000 95486.7800  11.7%  17.7  911s
 354020 164338 95554.0889   26  115 108159.000 95492.1711  11.7%  17.7  915s
 355297 164904 101121.102   32   98 108159.000 95497.5778  11.7%  17.7  920s
 357790 165948 106608.867   34   68 108159.000 95509.5578  11.7%  17.7  927s
 359079 166686 103400.587   33   88 108159.000 95515.1400  11.7%  17.7  931s
 360344 167277 107478.067   37   61 108159.000 95519.5022  11.7%  17.7  936s
 361586 167691     cutoff   40      108159.000 95519.7000  11.7%  17.7  940s
 364318 168335 97600.9933   32  121 108159.000 95531.2889  11.7%  17.7  948s
 365837 168986 96861.6178   28  118 108159.000 95535.9022  11.7%  17.7  952s
 366964 169442 101293.258   31   93 108159.000 95540.7956  11.7%  17.7  955s

...

  1263  1277 1301.92929  109   71          - 1090.97980      -  16.8   10s
  2123  2143 1390.61616  197   44          - 1090.97980      -  14.0   15s
  2976  2988 1503.52020  265   47          - 1090.97980      -  12.5   21s
H 3039  2988                    2511.0000000 1090.97980  56.6%  12.4   21s
H 3112  2988                    1939.0000000 1090.97980  43.7%  12.3   21s
H 3164  2993                    1581.0000000 1090.97980  31.0%  12.2   22s
H 3189  3073                    1573.0000000 1090.97980  30.6%  12.2   23s
H 3217  3000                    1507.0000000 1090.97980  27.6%  12.1   23s
H 3272  2981                    1506.0000000 1090.97980  27.6%  12.0   24s
H 3273  2974                    1504.0000000 1090.97980  27.5%  12.1   24s
H 3283  2916                    1475.0000000 1090.97980  26.0%  12.1   24s
H 3296  2928                    1474.0000000 1090.97980  26.0%  12.1   24s
  3325  3030 1135.69697   15  155 1474.00000 1094.95960  25.7%  12.2   25s
  4293  3897 1197.72121  

 102808 36111 1170.23299   29  136 1211.00000 1160.86735  4.14%  28.7  501s
 104188 36461 1162.11224   27  137 1211.00000 1160.93878  4.13%  28.7  506s
 105431 36895 1184.91327   31  137 1211.00000 1160.98980  4.13%  28.7  512s
 106814 37218 1185.14286   27  133 1211.00000 1161.07304  4.12%  28.8  517s
 108214 37558 1191.12500   31  143 1211.00000 1161.18027  4.11%  28.8  522s
 109513 37805     cutoff   35      1211.00000 1161.28571  4.11%  28.8  528s
 110836 38202 1206.45068   31  139 1211.00000 1161.41837  4.09%  28.8  534s
 112211 38790 1188.86224   28   93 1211.00000 1161.62925  4.08%  28.8  538s
 113571 39149 1197.54082   34  118 1211.00000 1161.78571  4.06%  28.8  543s
 114772 39539 1168.68878   25  109 1211.00000 1161.87415  4.06%  28.8  548s
 116145 39916 infeasible   34      1211.00000 1161.92857  4.05%  28.8  553s
 117492 40124     cutoff   33      1211.00000 1161.98980  4.05%  28.8  557s
 118682 40450 1169.08844   27  125 1211.00000 1162.07143  4.04%  28.8  562s
 120007 4077

 246850 71529 1199.75680   33  122 1211.00000 1168.40816  3.52%  28.7 1064s
 248173 71904 1192.48980   32   84 1211.00000 1168.50306  3.51%  28.7 1069s
 249455 72317 1195.83673   33  108 1211.00000 1168.57823  3.50%  28.7 1075s
 251961 73113 1200.40816   31  114 1211.00000 1168.67517  3.50%  28.7 1084s
 253185 73450 1195.89184   37  163 1211.00000 1168.69388  3.49%  28.7 1089s
 254420 73855 1202.56633   35  102 1211.00000 1168.72109  3.49%  28.7 1094s
 255678 74253     cutoff   31      1211.00000 1168.76531  3.49%  28.7 1099s
 257008 74602     cutoff   37      1211.00000 1168.79337  3.49%  28.7 1105s
 258228 74869     cutoff   37      1211.00000 1168.83878  3.48%  28.7 1110s
 259767 75134 1195.87585   29  155 1211.00000 1168.86735  3.48%  28.7 1118s
 260375 75542 1189.47959   31  143 1211.00000 1168.88776  3.48%  28.7 1126s
 262091 75815 1191.73469   34  124 1211.00000 1168.94388  3.47%  28.7 1133s
 263546 75975 1195.42347   31   83 1211.00000 1168.97449  3.47%  28.7 1140s
 264985 7626

 383365 100337 infeasible   34      1211.00000 1172.14116  3.21%  28.6 1844s
 384651 100556 1174.48469   29  142 1211.00000 1172.16327  3.21%  28.6 1854s
 385964 100734 1183.78061   31  106 1211.00000 1172.20255  3.20%  28.6 1863s
 387398 100913 infeasible   35      1211.00000 1172.23639  3.20%  28.6 1871s
 388625 101050     cutoff   31      1211.00000 1172.28061  3.20%  28.5 1880s
 389948 101188 infeasible   36      1211.00000 1172.34184  3.19%  28.6 1888s
 391252 101302     cutoff   42      1211.00000 1172.38776  3.19%  28.6 1896s
 392452 101501     cutoff   36      1211.00000 1172.43197  3.18%  28.6 1905s
 393813 101806 1188.62959   25  115 1211.00000 1172.47959  3.18%  28.6 1913s
 394977 102143     cutoff   33      1211.00000 1172.53367  3.18%  28.6 1922s
 396355 102533 1185.45918   29  137 1211.00000 1172.56686  3.17%  28.5 1929s
 397589 102813 1182.02551   30  144 1211.00000 1172.59576  3.17%  28.5 1937s
 398847 103107 1205.58673   32   83 1211.00000 1172.61967  3.17%  28.5 1946s

 519218 125692 1183.79592   31   94 1211.00000 1174.84184  2.99%  28.4 2758s
 520632 125988 1197.18537   30  138 1211.00000 1174.86429  2.98%  28.4 2768s
 521979 126248 1204.54762   35   60 1211.00000 1174.87415  2.98%  28.4 2778s
 523365 126496 infeasible   32      1211.00000 1174.89286  2.98%  28.4 2788s
 524752 126823 1176.26531   26  107 1211.00000 1174.91837  2.98%  28.4 2798s
 526148 126994 1181.00918   28  138 1211.00000 1174.93367  2.98%  28.4 2807s
 527567 127140 1179.52245   25  151 1211.00000 1174.95918  2.98%  28.4 2816s
 528952 127365 1203.68707   35   87 1211.00000 1174.97449  2.97%  28.4 2825s
 530141 127542     cutoff   29      1211.00000 1174.99320  2.97%  28.4 2834s
 531369 127796     cutoff   30      1211.00000 1175.01531  2.97%  28.4 2843s
 532722 127950     cutoff   32      1211.00000 1175.03486  2.97%  28.4 2852s
 534023 128158 1189.16837   29  119 1211.00000 1175.04082  2.97%  28.4 2861s
 535357 128308 infeasible   32      1211.00000 1175.06429  2.97%  28.4 2870s

In [18]:
# FORMULATION 3
params = {'MIPGap':0, 'TimeLimit':3600, 'Presolve':0, 'Cuts':0}
run_f3('3d_f3', params, datasets)

Set parameter Username
Set parameter LicenseID to value 2634667
Academic license - for non-commercial use only - expires 2026-03-10
Set parameter LogFile to value "logs/3d_f3_bays29.log"
Set parameter MIPGap to value 0
Set parameter TimeLimit to value 3600
Set parameter Presolve to value 0
Set parameter Cuts to value 0
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  3600
MIPGap  0
Cuts  0
Presolve  0

Optimize a model with 928 rows, 870 columns and 4064 nonzeros
Model fingerprint: 0xe20bd28c
Variable types: 0 continuous, 870 integer (841 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Variable types: 0 continuous, 870 integer (841 binary)

Ro

 138821 53870  671.95122   27   43  699.00000  650.89634  6.88%  10.2  130s
 146033 55369  670.42683   54   48  699.00000  651.43902  6.80%  10.3  136s
 150289 56210 infeasible   31       699.00000  651.78049  6.76%  10.4  140s
 156656 57349  663.90244   33   28  699.00000  652.34146  6.68%  10.5  146s
 160850 58095  688.42683   59   14  699.00000  652.75610  6.62%  10.5  151s
 165530 59105  655.43570   27   66  699.00000  653.16260  6.56%  10.6  155s
 171610 60316 infeasible   28       699.00000  653.61789  6.49%  10.7  161s
 176067 61064 infeasible   47       699.00000  654.07317  6.43%  10.7  165s
 182111 61921  664.97561   36   30  699.00000  654.47967  6.37%  10.8  170s
 186454 62415 infeasible   34       699.00000  654.90244  6.31%  10.9  175s
 193316 64363     cutoff   37       699.00000  655.26829  6.26%  10.9  180s
 199733 66086 infeasible   46       699.00000  655.51220  6.22%  10.8  185s
 206107 68160  696.03252   36    8  699.00000  655.70732  6.19%  10.8  190s
 212656 7012


    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 77656.2237    0  125 549932.000 77656.2237  85.9%     -    0s
H    0     0                    425743.00000 77656.2237  81.8%     -    1s
H    0     0                    351798.00000 77656.2237  77.9%     -    1s
H    0     0                    335420.00000 77656.2237  76.8%     -    1s
H    0     0                    201534.00000 77656.2237  61.5%     -    1s
     0     0 77656.2237    0  125 201534.000 77656.2237  61.5%     -    1s
H    0     0                    173657.00000 78991.2237  54.5%     -    1s
     0     2 78991.2237    0  125 173657.000 78991.2237  54.5%     -    1s
H   27    50                    173649.00000 78991.2237  54.5%  13.4    2s
   620   695 90733.7451   43  133 173649.000 78991.2237  54.5%  22.9    5s
H 1168  1178                    172350.00000 78991.2237  54.2%  17.0    7s
H 1172  1178           

 106946 54966     cutoff   38      108234.000 93777.3711  13.4%  17.4  326s
 109788 56254     cutoff   41      108234.000 93823.0400  13.3%  17.3  332s
 112414 57896 102512.113   33   84 108234.000 93857.2378  13.3%  17.3  336s
 115003 59095 107945.108   37   98 108234.000 93899.6933  13.2%  17.3  340s
 117639 60287 95240.3618   22  142 108234.000 93961.6178  13.2%  17.3  345s
 120222 61635 95825.3893   29  136 108234.000 93998.3956  13.2%  17.3  351s
 122928 63052 100335.018   32  104 108234.000 94032.1956  13.1%  17.3  355s
 127099 64998 101106.822   34  119 108234.000 94087.4800  13.1%  17.3  362s
 129889 66170 105097.658   38   65 108234.000 94132.2778  13.0%  17.3  366s
 132338 67441 99707.0733   29  111 108234.000 94156.6319  13.0%  17.3  370s
 136135 69209 104167.427   39   72 108234.000 94191.4067  13.0%  17.3  375s
 140099 71150 95136.8800   31  114 108234.000 94239.8844  12.9%  17.2  381s
 143773 73155 99369.7378   29  116 108234.000 94274.5000  12.9%  17.2  386s
 147516 7455

 333415 155526 101469.771   32  114 108159.000 95412.7689  11.8%  17.7  861s
 335913 156611     cutoff   38      108159.000 95421.4600  11.8%  17.7  867s
 337087 157229 102581.791   38  103 108159.000 95426.7378  11.8%  17.7  871s
 338426 157788 103941.861   35  129 108159.000 95431.7667  11.8%  17.7  875s
 341080 158521 99128.2311   33  120 108159.000 95443.8933  11.8%  17.7  882s
 341649 158754 103004.180   34  107 108159.000 95446.4756  11.8%  17.7  885s
 343553 159917 106027.358   37  134 108159.000 95452.8356  11.7%  17.7  893s
 344861 160563 97439.5289   30  131 108159.000 95455.8933  11.7%  17.7  897s
 346322 161139 104944.700   36  112 108159.000 95462.4933  11.7%  17.7  902s
 347749 161524 104773.847   37   76 108159.000 95468.9511  11.7%  17.7  906s
 348935 161966 104841.729   32  119 108159.000 95473.3878  11.7%  17.7  910s
 351465 163213 96492.2956   29  106 108159.000 95481.7333  11.7%  17.7  917s
 352720 163772 infeasible   33      108159.000 95486.7800  11.7%  17.7  921s

...

H 3283  2916                    1475.0000000 1090.97980  26.0%  12.1   24s
  3288  2934 1115.00000   14  167 1475.00000 1090.97980  26.0%  12.1   25s
H 3296  2928                    1474.0000000 1090.97980  26.0%  12.1   25s
  3917  3576 1169.59113   27  189 1474.00000 1096.32660  25.6%  12.6   30s
H 4470  3795                    1452.0000000 1096.67677  24.5%  12.8   32s
H 4472  3607                    1450.0000000 1096.67677  24.4%  12.8   33s
H 4474  3427                    1231.0000000 1096.67677  10.9%  12.8   34s
H 4474  3256                    1223.0000000 1096.67677  10.3%  12.8   34s
H 4474  3093                    1220.0000000 1096.67677  10.1%  12.8   34s
H 4474  2938                    1212.0000000 1096.67677  9.52%  12.8   34s
  4476  2942 1105.91837   12  169 1212.00000 1105.91837  8.75%  13.0   36s
  4689  3095 1166.47619   24  138 1212.00000 1105.91837  8.75%  14.1   40s
  5146  3278 1206.80850   40  134 1212.00000 1105.91837  8.75%  15.3   45s
H 5274  3153             

 110836 38202 1206.45068   31  139 1211.00000 1161.41837  4.09%  28.8  556s
 112211 38790 1188.86224   28   93 1211.00000 1161.62925  4.08%  28.8  562s
 113571 39149 1197.54082   34  118 1211.00000 1161.78571  4.06%  28.8  566s
 114772 39539 1168.68878   25  109 1211.00000 1161.87415  4.06%  28.8  572s
 116145 39916 infeasible   34      1211.00000 1161.92857  4.05%  28.8  577s
 117492 40124     cutoff   33      1211.00000 1161.98980  4.05%  28.8  582s
 118682 40450 1169.08844   27  125 1211.00000 1162.07143  4.04%  28.8  587s
 120007 40773 infeasible   34      1211.00000 1162.15028  4.03%  28.8  592s
 121244 41160 1171.18571   27  136 1211.00000 1162.23588  4.03%  28.8  596s
 122410 41501 1198.83469   32  137 1211.00000 1162.29025  4.02%  28.8  601s
 123584 41827 1172.30612   30  112 1211.00000 1162.38980  4.01%  28.8  606s
 124952 42160 1171.12755   27  129 1211.00000 1162.57535  4.00%  28.9  612s
 126217 42548 1180.76735   29  127 1211.00000 1162.75510  3.98%  28.8  616s
 127385 4305

 249455 72317 1195.83673   33  108 1211.00000 1168.57823  3.50%  28.7 1161s
 250748 72644 1201.63152   34  145 1211.00000 1168.63265  3.50%  28.7 1166s
 251961 73113 1200.40816   31  114 1211.00000 1168.67517  3.50%  28.7 1173s
 253185 73450 1195.89184   37  163 1211.00000 1168.69388  3.49%  28.7 1178s
 254420 73855 1202.56633   35  102 1211.00000 1168.72109  3.49%  28.7 1184s
 255678 74253     cutoff   31      1211.00000 1168.76531  3.49%  28.7 1191s
 257008 74602     cutoff   37      1211.00000 1168.79337  3.49%  28.7 1197s
 258228 74869     cutoff   37      1211.00000 1168.83878  3.48%  28.7 1203s
 259521 74911 1183.08418   28  159 1211.00000 1168.86735  3.48%  28.7 1208s
 259767 75134 1195.87585   29  155 1211.00000 1168.86735  3.48%  28.7 1213s
 260375 75542 1189.47959   31  143 1211.00000 1168.88776  3.48%  28.7 1222s
 262091 75815 1191.73469   34  124 1211.00000 1168.94388  3.47%  28.7 1229s
 263546 75975 1195.42347   31   83 1211.00000 1168.97449  3.47%  28.7 1237s
 264985 7626

 381955 100087 1199.81122   29  163 1211.00000 1172.10612  3.21%  28.6 1997s
 383365 100337 infeasible   34      1211.00000 1172.14116  3.21%  28.6 2006s
 384651 100556 1174.48469   29  142 1211.00000 1172.16327  3.21%  28.6 2017s
 385964 100734 1183.78061   31  106 1211.00000 1172.20255  3.20%  28.6 2027s
 387398 100913 infeasible   35      1211.00000 1172.23639  3.20%  28.6 2036s
 388625 101050     cutoff   31      1211.00000 1172.28061  3.20%  28.5 2045s
 389948 101188 infeasible   36      1211.00000 1172.34184  3.19%  28.6 2054s
 391252 101302     cutoff   42      1211.00000 1172.38776  3.19%  28.6 2063s
 392452 101501     cutoff   36      1211.00000 1172.43197  3.18%  28.6 2073s
 393813 101806 1188.62959   25  115 1211.00000 1172.47959  3.18%  28.6 2081s
 394977 102143     cutoff   33      1211.00000 1172.53367  3.18%  28.6 2091s
 396355 102533 1185.45918   29  137 1211.00000 1172.56686  3.17%  28.5 2099s
 397589 102813 1182.02551   30  144 1211.00000 1172.59576  3.17%  28.5 2108s

 518009 125257     cutoff   32      1211.00000 1174.82653  2.99%  28.4 2998s
 518754 125360 1192.61224   31  107 1211.00000 1174.83673  2.99%  28.4 3004s
 519218 125692 1183.79592   31   94 1211.00000 1174.84184  2.99%  28.4 3015s
 520632 125988 1197.18537   30  138 1211.00000 1174.86429  2.98%  28.4 3026s
 521979 126248 1204.54762   35   60 1211.00000 1174.87415  2.98%  28.4 3037s
 523365 126496 infeasible   32      1211.00000 1174.89286  2.98%  28.4 3048s
 524752 126823 1176.26531   26  107 1211.00000 1174.91837  2.98%  28.4 3058s
 526148 126994 1181.00918   28  138 1211.00000 1174.93367  2.98%  28.4 3069s
 527567 127140 1179.52245   25  151 1211.00000 1174.95918  2.98%  28.4 3079s
 528952 127365 1203.68707   35   87 1211.00000 1174.97449  2.97%  28.4 3090s
 530141 127542     cutoff   29      1211.00000 1174.99320  2.97%  28.4 3099s
 531369 127796     cutoff   30      1211.00000 1175.01531  2.97%  28.4 3110s
 532722 127950     cutoff   32      1211.00000 1175.03486  2.97%  28.4 3119s